<a href="https://colab.research.google.com/github/Elmir22/Data-science-ML-models-/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/main/helper_functions.py

--2024-05-13 10:07:11--  https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-13 10:07:11 (59.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from  helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir, compare_historys
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-05-13 10:12:25--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.2’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2024-05-13 10:12:25 (86.5 MB/s) - ‘nlp_getting_started.zip.2’ saved [607343/607343]



In [10]:
#turn .csv files into pandas DataFrame"s
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


#the traning data we downloaded is probably shuffled already

In [11]:
#suflle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [12]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
#how many examples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [15]:
print(f"total traning samples : {len(train_df)}")
print(f"total test samples : {len(test_df)}")
print(f"total samples : {len(train_df) + len(test_df)}")

total traning samples : 7613
total test samples : 3263
total samples : 10876


In [16]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}", "(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target:1 (real disaster)
Text:
Tension In Bayelsa As Patience Jonathan Plans To Hijack APC PDP http://t.co/qxXN6RKsp6 http://t.co/B3X1wqzAoR

---

Target:1 (real disaster)
Text:
#???? #?? #??? #??? Trains derailment: 'It's the freakiest of freak accidents' - The Indian Express   http://t.co/4Y4YtwhD74

---

Target:1 (real disaster)
Text:
Chevrolet : Avalanche LT 2011 lt used 5.3 l v 8 16 v automatic 4 wd pickup truck premium bÛ_ http://t.co/OBkY8Pc89H http://t.co/dXIRnTdSrd

---

Target:1 (real disaster)
Text:
#TRAFFICALERT  Eastbound 210 Freeway at Citrus Ave in Azusa. Two motorcycles involved in accident with one fully engulfed in flames in lanes

---

Target:0 (not real disaster)
Text:
@burberryant bleeding on the brain don't know the cause

---



In [17]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [18]:
len(train_sentences), len(train_labels), len(val_labels), len(val_sentences)

(6851, 6851, 762, 762)

In [19]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization #after Tensorflow 2.6

text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocaulary ( all of the different word in your text)
                                    standardize="lower_and_strip_punctuation",# how to process text
                                    split="whitespace",#how to split tokens
                                    ngrams=None,#create groups of n-word,
                                    output_mode="int",# how to map tokens to numbers
                                    output_sequence_length=None)# how long  should the output
                                    #pad_to_max_tokens=True) # not valid if using max_tokens=None

In [22]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

#Now lets create another TextVectorization object using our custom parametrs.

In [23]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [24]:
#fit the text vectorizer to the traning text
text_vectorizer.adapt(train_sentences)

In [29]:
#Create sample sentence and tokenize it
sample_sentence = "there's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,  15,   1,   0,   0,   0,   0,   0,
          0,   0]])>

In [30]:
#choose a random sentence from the traning dataset
random_sentence = random.choice(train_sentences)
print(f"Original text :\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text :
The cryptic words that guided pilots on the Hiroshima bombing mission http://t.co/39IAbcC5pK      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 3959,  590,   16, 3798, 2021,   11,    2,  130,  156, 2042,
           1,    0,    0,    0]])>

In [ ]:
#get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab : {len(words_in_vocab)}")
print(f"top 5 most common words : {top_5_words}")
print(f"bottom 5 least common words : {bottom_5_words}")


In [32]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # set size of embedding vector
                             embeddings_initializer= "uniform",# default , initialize randomly
                             input_length = max_length,# how long is each input
                             name= "embedding_1")
embedding

In [33]:
#get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
       \n\nEmbedded version:")
#embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
627% but if they had lower striked than 16 I would have gone even further OTM. This could really fall off a cliff.       

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03459921, -0.03416977, -0.00952493, ..., -0.01669895,
          0.02280006,  0.00358834],
        [-0.01015569,  0.01901923,  0.04971651, ...,  0.03790721,
          0.02284428,  0.01686061],
        [ 0.03296447, -0.01059934,  0.04895026, ..., -0.03113496,
          0.0082804 , -0.04864449],
        ...,
        [ 0.001842  , -0.00889869, -0.00332318, ..., -0.04334651,
          0.01613781,  0.04473957],
        [-0.02024006, -0.04101809,  0.03647559, ...,  0.02908205,
         -0.04033583, -0.00906916],
        [-0.01289953, -0.01377898,  0.00036835, ..., -0.04220475,
          0.02681864, -0.04816112]]], dtype=float32)>

In [34]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03459921, -0.03416977, -0.00952493, -0.03491669, -0.0441173 ,
       -0.01669963,  0.01723936, -0.00634896,  0.02319827,  0.03857053,
       -0.00400562,  0.01556252, -0.03452003,  0.04343145,  0.00049524,
       -0.00194827,  0.01124762, -0.00569869,  0.03752852,  0.02748556,
        0.0228956 , -0.04204699, -0.0290934 , -0.02687748, -0.0475032 ,
       -0.0479675 , -0.00310782,  0.01601933, -0.04249798, -0.03306024,
       -0.0283055 ,  0.02993455, -0.01129258, -0.02468933, -0.03750848,
        0.01767746, -0.01031461, -0.01716808,  0.02380871, -0.04931251,
        0.02713032, -0.04994015, -0.01441294,  0.03216212, -0.04516682,
       -0.02954753, -0.03612412,  0.04508854,  0.01490532, -0.01141995,
       -0.02874789,  0.01746793,  0.01479151, -0.00937103, -0.00210167,
        0.03903538, -0.03013517, -0.04174124, -0.04812504, -0.02227733,
        0.02985409, -0.04142159, -0.01136894,  0.00299226, -0.01577488,
        0.008679

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfiddf
    ("clf", MultinomialNB()) # model the text
])
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])